In [ ]:
# Colab 에 Mecab 설치

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import urllib.request
import pandas as pd

# traing & test set download, save
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [ ]:
train_df.head() # 상위 5개의 정보 & col 정보를 가져온다

In [ ]:
test_df.head()

In [ ]:
print('훈련 데이터 샘플의 개수 : {}'.format(len(train_df)))
print('테스트 데이터 샘플의 개수 : {}'.format(len(test_df)))

In [ ]:
# 필드 정의하기

from torchtext import data # torchtext.data 임포트
from konlpy.tag import Mecab

In [ ]:
# Mecab 을 토크나이저로 사용한다
tokenizer = Mecab()

In [ ]:
ID = data.Field(sequential = False,
                use_vocab = False)

TEXT = data.Field(sequential = True,
                  use_vocab = True,
                  tokenize = tokenizer.morphs,
                  lower = True,
                  batch_first = True,
                  fix_length = 20)

LABEL = data.Field(sequential = False,
                   use_vocab = False,
                   is_target = True)

# sequential : 시퀀스 데이터 여부. (True가 기본값)
# use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
# tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
# lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
# batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
# is_target : 레이블 데이터 여부. (False가 기본값)
# fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [ ]:
# 데이터 셋 만들기

from torchtext.data import TabularDataset

train_data, test_data = TabularDataset.splits(
    path = '.', train = 'ratings_train.txt', test = 'ratings_test.txt', format = 'tsv',
    fields = [('id',ID), ('text', TEXT), ('label', LABEL)], skip_header = True
)

In [ ]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

In [ ]:
print(vars(train_data[0]))

In [ ]:
# vocabulary (단어 집합)

TEXT.build_vocab(train_data, min_freq = 10, max_size = 10000)
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

print(TEXT.vocab.stoi) # iterator 로 표현

In [ ]:
# 토치 텍스트의 데이터 로더 만들기

from torchtext.data import Iterator

batch_size = 5
train_loader = Iterator(dataset = train_data, batch_size = batch_size)
test_loader = Iterator(dataset = test_data, batch_size = batch_size)

print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

In [ ]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text)

In [ ]:
batch = next(iter(train_loader)) # 두번째 미니배치
print(batch.text)

In [ ]:
# 토치 텍스트에서 batch_first True / False 비교

import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

In [ ]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
print('전체 샘플의 개수 : {}'.format(len(df)))

In [ ]:
train_df = df[:25000] # slicing 은 얕은 복사에 해당한다
test_df = df[25000:]

In [ ]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [ ]:
from torchtext import data

TEXT = data.Field(sequential=True,
                  use_vocab = True,
                  tokenize = str.split,
                  lower = True,
                  batch_first = True, # 미니배치 차원을 맨 앞으로 해서 데이터를 불러올 것인지 여부,
                  fix_length = 20)


LABEL = data.Field(sequential = False,
                   use_vocab = False,
                   batch_first = False,
                   is_target = True)

In [ ]:
from torchtext.data import TabularDataset
from torchtext.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠습니다.
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text)

In [ ]:
print(batch.text.shape)

In [ ]:
# 필드 재정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)


In [ ]:
print(batch.text)